In [10]:
import importlib  
imageprocessing = importlib.import_module("JV-ImagesProcessingFunctions")
import pandas as pd
import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join
import tensorflow as tf
import cv2
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Model
from tensorflow.keras.layers import Input,Activation,Add, MaxPooling2D, MaxPooling1D, Flatten, Dense, Conv1D, Dropout
import gc
from ipynb.fs.full.SIFT_to_Features import SIFT_nparray_to_Features
from tensorflow import keras
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix

In [11]:
#Simplifies adding a layer
def Convolution(input_tensor,filters):

    x = layers.Conv2D(filters=filters,kernel_size=(3, 3),padding = 'same', activation = 'relu')(input_tensor)

    return x

def Convolution1D(input_tensor,filters):

    x = layers.Conv1D(filters=filters,kernel_size=(3),padding = 'same', activation = 'relu')(input_tensor)

    return x

#Simple CNN model
#Conv stands for neural network layers of (x,y). x standing for the input layers and y standing for how many filter layers there are
#maxp is a pooling layer that reduces the number of nodes of the next layer
#flatten changes the previous layer to a 1 dimensional layer
#dense layers have each node connected to every node in the previous layer. In this case, we have 3 dense layers of differing weights representing
#age, gender, and race. While all previous layers have collected features, dense layers calculate based on these collected features.
#They then send their calculations to the final dense layer to determine classification
#Binary Crossentropy = Classifies based on a binary value
#Sparse Categorical Crossentropy = Classifies based on an integer value with more than 2 possible values
def Simplemodel(input_shape,column):
  losstype = ""
  outputn = 0
  if(column == 'age'):
    losstype = "sparse_categorical_crossentropy"
    outputn = 9
  if(column == 'gender'):
    losstype = "binary_crossentropy"
    outputn = 1
  if(column == 'race'):
    losstype = "sparse_categorical_crossentropy"
    outputn = 5


  inputs = Input((input_shape))

  conv_1= Convolution(inputs,32)

  maxp_1 = MaxPooling2D(pool_size = (2,2)) (conv_1)

  conv_2 = Convolution(maxp_1,64)

  maxp_2 = MaxPooling2D(pool_size = (2, 2)) (conv_2)

  conv_3 = Convolution(maxp_2,64)

  maxp_3 = MaxPooling2D(pool_size = (2, 2)) (conv_3)

  conv_4 = Convolution(maxp_3,64)

  maxp_4 = MaxPooling2D(pool_size = (2, 2)) (conv_4)

  conv_5 = Convolution(maxp_4,64)

  flatten= Flatten() (conv_5)

  dense_1= Dense(64,activation='relu')(flatten)

  output_1= Dense(outputn,activation="sigmoid",name='out')(dense_1)

  model = Model(inputs=[inputs], outputs=[output_1])

  model.compile(loss=[losstype], optimizer="Adam",

  metrics=["accuracy"])

  return model

def Simplemodel1D(input_shape,column):
  losstype = ""
  outputn = 0
  if(column == 'age'):
    losstype = "sparse_categorical_crossentropy"
    outputn = 9
  if(column == 'gender'):
    losstype = "binary_crossentropy"
    outputn = 1
  if(column == 'race'):
    losstype = "sparse_categorical_crossentropy"
    outputn = 5

  inputs = Input((input_shape))

  conv_1= Convolution1D(inputs,32)

  maxp_1 = MaxPooling1D(pool_size = 2) (conv_1)

  conv_2 = Convolution1D(maxp_1,64)

  maxp_2 = MaxPooling1D(pool_size = 2) (conv_2)

  conv_3 = Convolution1D(maxp_2,64)

  maxp_3 = MaxPooling1D(pool_size = 2) (conv_3)

  conv_4 = Convolution1D(maxp_3,64)

  maxp_4 = MaxPooling1D(pool_size = 2) (conv_4)

  conv_5 = Convolution1D(maxp_4,64)

  flatten= Flatten() (conv_5)

  dense_1= Dense(64,activation='relu')(flatten)

  output_1= Dense(outputn,activation="sigmoid",name='out')(dense_1)

  model = Model(inputs=[inputs], outputs=[output_1])

  model.compile(loss=[losstype], optimizer="Adam",

  metrics=["accuracy"])

  return model

In [12]:
# Get data and convert for model
data = np.load('../data/raw/UnbalancedRaw/npimagearrays.npy',allow_pickle=True)
filelabels = pd.read_csv('../data/raw/UnbalancedRaw/npimagefilelabels.csv').loc[:,['file','age']]

X_train, X_test, y_train, y_test = train_test_split(data, filelabels, test_size=0.33, random_state=42)

y_testnp = np.asarray(y_test.loc[:,'age'])

y_trainnp = np.asarray(y_train.loc[:,'age'])

In [13]:
def SelectMask(nparraytrain,nparraytest,select = 0):
    """
    No mask option Select = [1< or >10]
    """
    X_train_mask = nparraytrain
    X_test_mask = nparraytest
    """
    Sobel filter on image picture channel
    """
    if(select == 1):
        X_train_mask = imageprocessing.sobelFilterTypes(nparraytrain)[0]
        X_test_mask = imageprocessing.sobelFilterTypes(nparraytest)[0]
    """
    Sobel filter on image picture hsv
    """
    if(select == 2):
        X_train_mask = imageprocessing.sobelFilterTypes(nparraytrain)[1]
        X_test_mask = imageprocessing.sobelFilterTypes(nparraytest)[1]
    """
    returns two numpy arrays with pictures in black and white and with the edges define using Roberts filter
    """
    if(select == 3):
        X_train_mask = imageprocessing.robertsSobelEdges(nparraytrain)[0]
        X_test_mask = imageprocessing.robertsSobelEdges(nparraytest)[0]
    """
    returns two numpy arrays with pictures in black and white and with the edges define using Sobel filter
    """
    if(select == 4):
        X_train_mask = imageprocessing.robertsSobelEdges(nparraytrain)[1]
        X_test_mask = imageprocessing.robertsSobelEdges(nparraytest)[1]
    """
    This fucntion increases the exposure of each picture and it returns a numpy
    array of the modified images. The images are in color
    """
    if(select == 5):
        X_train_mask = imageprocessing.exposure_(nparraytrain)
        X_test_mask = imageprocessing.exposure_(nparraytest)

    """
    This function gamma corrects the image
    """
    if(select == 6):
        X_train_mask = imageprocessing.gammaLogaritmictCorrection(nparraytrain)[0]
        X_test_mask = imageprocessing.gammaLogaritmictCorrection(nparraytest)[0]
    """
    This function logarithmic corrects the image
    """
    if(select == 7):
        X_train_mask = imageprocessing.gammaLogaritmictCorrection(nparraytrain)[1]
        X_test_mask = imageprocessing.gammaLogaritmictCorrection(nparraytest)[1]
    """
    This function rescales the image
    """
    if(select == 8):
        X_train_mask = imageprocessing.histogramEqualizer(nparraytrain)[0]
        X_test_mask = imageprocessing.histogramEqualizer(nparraytest)[0]
    """
    This function equalizes the image
    """
    if(select == 9):
        X_train_mask = imageprocessing.histogramEqualizer(nparraytrain)[1]
        X_test_mask = imageprocessing.histogramEqualizer(nparraytest)[1]
    """
    This function adapthis the image
    """
    if(select == 10):
        X_train_mask = imageprocessing.histogramEqualizer(nparraytrain)[2]
        X_test_mask = imageprocessing.histogramEqualizer(nparraytest)[2]
    return X_train_mask,X_test_mask

In [14]:
def Preprocessnparrays(X_train,
                       X_test,
                       y_train,
                       y_test,
                       mask = 0,
                       sift = False,
                       vocab = 200,
                       pca = False,
                       n_components = 200 ,
                       rfs = False,
                       n_estimators = 20):
    X_train, X_test = SelectMask(X_train,X_test,select = mask)
    if(sift):
        X_train, X_test = SIFT_nparray_to_Features(X_train,X_test,nvocab = vocab)
        if(pca):
            pcax = PCA(n_components = n_components)
            pcax.fit(X_train)
            X_train = pcax.transform(X_train)
            X_test = pcax.transform(X_test)
        if(rfs):
            selector = SelectFromModel(RandomForestClassifier(n_estimators = n_estimators))
            selector.fit(X_train, y_train)
            X_train = X_train[:,selector.get_support()]
            X_test = X_test[:,selector.get_support()]
    else:
        if(pca or rfs):
            X_train = X_train.reshape(len(X_train),200*200*3)
            X_test = X_test.reshape(len(X_test),200*200*3)
            if(pca):
                pcax = PCA(n_components = n_components)
                pcax.fit(X_train)
                X_train = pcax.transform(X_train)
                X_test = pcax.transform(X_test)
            if(rfs):
                selector = SelectFromModel(RandomForestClassifier(n_estimators = n_estimators))
                selector.fit(X_train, y_train)
                X_train = X_train[:,selector.get_support()]
                X_test = X_test[:,selector.get_support()]
    return X_train,X_test,y_train,y_test

In [ ]:
#Creating the model
Modelsimple=Simplemodel((200,200,3),'age')

In [ ]:
Modelsift=Simplemodel1D((200,1))
Modelsift.summary()

In [15]:
confusionmatrices = []
histories = []

In [16]:
X_train_new,X_test_new,y_train_new,y_test_new = Preprocessnparrays(X_train[:1000],
                                                                       X_test[:100],
                                                                       y_trainnp[:1000],
                                                                       y_testnp[:100],
                                                                       mask = 0,
                                                                       sift = True,
                                                                       vocab = 200,
                                                                       pca = False,
                                                                       n_components = 200 ,
                                                                       rfs = False,
                                                                       n_estimators = 20)

In [155]:
tf.expand_dims(X_train_new, axis=-1)[0]

<tf.Tensor: shape=(200, 200, 1), dtype=float64, numpy=
array([[[0.01616904],
        [0.02218374],
        [0.02235638],
        ...,
        [0.01753779],
        [0.00620054],
        [0.01568627]],

       [[0.02660279],
        [0.02529322],
        [0.02146884],
        ...,
        [0.02772968],
        [0.02111829],
        [0.00392157]],

       [[0.02772968],
        [0.02616013],
        [0.02111829],
        ...,
        [0.03100272],
        [0.04340381],
        [0.00392157]],

       ...,

       [[0.09835244],
        [0.1464169 ],
        [0.11154088],
        ...,
        [0.02777107],
        [0.03970286],
        [0.03529412]],

       [[0.08041607],
        [0.13476737],
        [0.11846124],
        ...,
        [0.04544435],
        [0.0570989 ],
        [0.05882353]],

       [[0.07058824],
        [0.1254902 ],
        [0.11764706],
        ...,
        [0.01148196],
        [0.01176471],
        [0.        ]]])>

In [22]:
for i in range(11):
    print(i)
    X_train_new,X_test_new,y_train_new,y_test_new = Preprocessnparrays(X_train[:1000],
                                                                       X_test[:100],
                                                                       y_trainnp[:1000],
                                                                       y_testnp[:100],
                                                                       mask = 0,
                                                                       sift = True,
                                                                       vocab = 200,
                                                                       pca = True,
                                                                       n_components = 200 ,
                                                                       rfs = True,
                                                                       n_estimators = 20)
    
    #Creating the model
    Type = 'age'
    Modelsimple = ''
    if(len(X_train_new.shape)==3):
        X_train_new = tf.expand_dims(X_train_new, axis=-1)
        X_test_new = tf.expand_dims(X_test_new, axis=-1)
        Modelsimple=Simplemodel(X_train_new[0].shape,Type)
    elif(len(X_train_new.shape)==2):
        X_train_new = tf.expand_dims(X_train_new, axis=-1)
        X_test_new = tf.expand_dims(X_test_new, axis=-1)
        Modelsimple=Simplemodel1D(X_train_new[0].shape,Type)
    else:
        Modelsimple=Simplemodel(X_train_new[0].shape,Type)
    
    if(Type == 'age'):
       y_test_new = y_test_new - 1
       y_train_new = y_train_new - 1
    
    History = Modelsimple.fit(X_train_new,y_train_new,validation_data=(X_test_new,y_test_new),epochs=10)
    History.history
    
    histories.append(History.history)
    
    ypred = Modelsimple.predict(X_test_new)
    
    confusionmatrices.append(confusion_matrix(np.argmax(ypred, axis=-1),y_test_new))

False
Epoch 1/10
32/32 [==============================] - 1s 7ms/step - loss: 37.0466 - accuracy: 0.2840 - val_loss: 9.8169 - val_accuracy: 0.2900
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 5.5419 - accuracy: 0.3280 - val_loss: 5.0597 - val_accuracy: 0.3000
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 2.8647 - accuracy: 0.3870 - val_loss: 3.2466 - val_accuracy: 0.2900
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 2.2009 - accuracy: 0.4220 - val_loss: 2.8888 - val_accuracy: 0.3500
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 1.7640 - accuracy: 0.4680 - val_loss: 2.7858 - val_accuracy: 0.3700
Epoch 6/10
32/32 [==============================] - 0s 5ms/step - loss: 1.4522 - accuracy: 0.5360 - val_loss: 2.6538 - val_accuracy: 0.3600
Epoch 7/10
32/32 [==============================] - 0s 5ms/step - loss: 1.2387 - accuracy: 0.5660 - val_loss: 2.7030 - val_accuracy: 0.3200
Epoch 8/10
32